# ResNet

![ResNet](https://pic4.zhimg.com/v2-4fb840788867722ba48fb6398041c678_1200x500.jpg)

In [1]:
# An implementation of https://arxiv.org/pdf/1512.03385.pdf  
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters
num_epochs = 80
learning_rate = 0.001
batch_size = 100

# Image preprocessing pipeline
# Possible preprocess: https://zhuanlan.zhihu.com/p/53367135
transform = transforms.Compose([transforms.Pad(4), 
                                transforms.RandomHorizontalFlip(), 
                                transforms.RandomCrop(32), # CIFAR10 dataset: 32*32 images
                                transforms.ToTensor()])  # covert PIL image or ndarray to tensor and normal to 0-1

# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                             train=True,
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='../data', 
                                            train=False,
                                            transform=transforms.ToTensor())  # No preprocess for test data

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [3]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual # Skip connection / shortcut connection
        out = self.relu(out)
        return out

# Resnet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)  # kernel_size=8
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):  # If need to perform downsample
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels  # After downsample
        for i in range(1, blocks):  # Add blocks to the layer
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)  # Unpack list
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)  # view: concat tensor rows to one row
        out = self.fc(out)
        return out
        
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# Loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate decay
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [4]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /=3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.6156
Epoch [1/80], Step [200/500] Loss: 1.3994
Epoch [1/80], Step [300/500] Loss: 1.2138
Epoch [1/80], Step [400/500] Loss: 1.1255
Epoch [1/80], Step [500/500] Loss: 1.1979
Epoch [2/80], Step [100/500] Loss: 1.1656
Epoch [2/80], Step [200/500] Loss: 1.1946
Epoch [2/80], Step [300/500] Loss: 1.0942
Epoch [2/80], Step [400/500] Loss: 1.0940
Epoch [2/80], Step [500/500] Loss: 0.9612
Epoch [3/80], Step [100/500] Loss: 0.6714
Epoch [3/80], Step [200/500] Loss: 0.8804
Epoch [3/80], Step [300/500] Loss: 0.8104
Epoch [3/80], Step [400/500] Loss: 0.8613
Epoch [3/80], Step [500/500] Loss: 0.7888
Epoch [4/80], Step [100/500] Loss: 0.8786
Epoch [4/80], Step [200/500] Loss: 0.7210
Epoch [4/80], Step [300/500] Loss: 0.7980
Epoch [4/80], Step [400/500] Loss: 0.8328
Epoch [4/80], Step [500/500] Loss: 0.6974
Epoch [5/80], Step [100/500] Loss: 0.8466
Epoch [5/80], Step [200/500] Loss: 0.5709
Epoch [5/80], Step [300/500] Loss: 0.6960
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.1948
Epoch [39/80], Step [400/500] Loss: 0.3710
Epoch [39/80], Step [500/500] Loss: 0.3155
Epoch [40/80], Step [100/500] Loss: 0.2908
Epoch [40/80], Step [200/500] Loss: 0.2381
Epoch [40/80], Step [300/500] Loss: 0.3984
Epoch [40/80], Step [400/500] Loss: 0.1643
Epoch [40/80], Step [500/500] Loss: 0.3065
Epoch [41/80], Step [100/500] Loss: 0.1554
Epoch [41/80], Step [200/500] Loss: 0.2349
Epoch [41/80], Step [300/500] Loss: 0.3119
Epoch [41/80], Step [400/500] Loss: 0.1090
Epoch [41/80], Step [500/500] Loss: 0.2400
Epoch [42/80], Step [100/500] Loss: 0.2384
Epoch [42/80], Step [200/500] Loss: 0.1865
Epoch [42/80], Step [300/500] Loss: 0.2418
Epoch [42/80], Step [400/500] Loss: 0.2582
Epoch [42/80], Step [500/500] Loss: 0.1964
Epoch [43/80], Step [100/500] Loss: 0.0830
Epoch [43/80], Step [200/500] Loss: 0.1683
Epoch [43/80], Step [300/500] Loss: 0.1707
Epoch [43/80], Step [400/500] Loss: 0.1550
Epoch [43/80], Step [500/500] Loss: 0.1514
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.0701
Epoch [77/80], Step [500/500] Loss: 0.1807
Epoch [78/80], Step [100/500] Loss: 0.1315
Epoch [78/80], Step [200/500] Loss: 0.1468
Epoch [78/80], Step [300/500] Loss: 0.1073
Epoch [78/80], Step [400/500] Loss: 0.2093
Epoch [78/80], Step [500/500] Loss: 0.2238
Epoch [79/80], Step [100/500] Loss: 0.1459
Epoch [79/80], Step [200/500] Loss: 0.0665
Epoch [79/80], Step [300/500] Loss: 0.1579
Epoch [79/80], Step [400/500] Loss: 0.1527
Epoch [79/80], Step [500/500] Loss: 0.0536
Epoch [80/80], Step [100/500] Loss: 0.1921
Epoch [80/80], Step [200/500] Loss: 0.1429
Epoch [80/80], Step [300/500] Loss: 0.1006
Epoch [80/80], Step [400/500] Loss: 0.1436
Epoch [80/80], Step [500/500] Loss: 0.1051


In [5]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

Accuracy of the model on the test images: 88.5 %
